In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型花园 - ImageBind

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_imagebind.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_imagebind.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_imagebind.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在Vertex AI Workbench中打开
    </a>（建议使用Python-3 CPU笔记本）
  </td>
</table>

## 概述

本笔记本演示了在Vertex AI中部署预构建的Imagebind模型进行在线预测。

### 目标

- 将ImageBind模型上传到[Vertex AI模型注册表](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)。
- 将ImageBind部署到[Vertex AI端点资源](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints)。
- 运行在线预测以生成特征嵌入和零样本分类。

### 成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

## 开始之前

**注意**: Jupyter会将前缀为`!`的行作为shell命令运行，并且会将前缀为`$`的Python变量插入到这些命令中。

只能使用 Colab
如果您使用的是工作台，请跳过此部分，并运行以下命令。

In [ ]:
import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### 设置 Google Cloud 项目

1. [选择或创建 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得300美元的免费信用额，可用于您的计算/存储成本。

2. [确保项目已启用计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API 和 Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component.googleapis.com)。

4. [创建一个云存储桶](https://cloud.google.com/storage/docs/creating-buckets)，用于存储实验结果。

5. [创建一个服务帐号](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console)，具有“Vertex AI用户”和“存储对象管理员”角色，以将优化后的模型部署到 Vertex AI 端点。

设置实验环境的以下变量。指定的云存储桶（BUCKET_URI）应该位于指定的区域（REGION）。请注意，一个多区域的存储桶（例如“us”）不被视为与多区域范围覆盖的单个区域（例如“us-central1”）匹配。

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = ""  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output.
# Start with gs:// prefix, e.g. gs://foo_bucket.
BUCKET_URI = "gs://"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID

import os

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
DATA_BUCKET = os.path.join(BUCKET_URI, "data")

# The service account looks like:
# '@.iam.gserviceaccount.com'
# Please go to https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create service account with `Vertex AI User` and `Storage Object Admin` roles.
# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

### 初始化 Vertex AI API

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

### 定义常量

In [ ]:
# The pre-built serving docker image.
PREDICTION_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-imagebind-serve"

定义常见功能

In [ ]:
import os
from datetime import datetime

import numpy as np


def get_job_name_with_datetime(prefix: str) -> str:
    """Gets the job name with date time when triggering deployment jobs."""
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def deploy_model(
    model_name: str,
    service_account: str,
    task: str,
    machine_type: str = "g2-standard-8",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: str = 1,
) -> tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys prebuilt model in Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-{task}-endpoint")
    serving_env = {
        "MODEL_ID": "ImageBind-feature-embedding-generation-001",
        "TASK": task,
        "DEPLOY_SOURCE": "notebook",
    }
    model = aiplatform.Model.upload(
        display_name=f"{model_name}-{task}",
        serving_container_image_uri=PREDICTION_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/imagebind_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    return model, endpoint

部署预构建的ImageBind模型

本部分在Vertex AI端点上部署预构建的ImageBind模型，用于特征嵌入生成和零-shot分类任务。模型部署步骤将需要大约15分钟才能完成。

In [ ]:
# Prepares example input data.
! git clone https://github.com/facebookresearch/ImageBind.git
%cd ImageBind/.assets
! git reset --hard 95d27c7fd5a8362f3527e176c3a80ae5a4d880c0

! gsutil cp -r . $DATA_BUCKET

%cd ../..

### 部署预构建的 ImageBind 模型用于特征嵌入生成

在本节中，我们部署一个 ImageBind，用于为不同的数据模态生成特征嵌入。

ImageBind 模型的峰值 GPU 内存使用量约为 8G。请相应调整机器类型、加速器类型和加速器数量。我们在部署中使用一个 L4（24G）作为示例。

In [ ]:
task = "feature-embedding-generation"

In [ ]:
# Finds Vertex AI prediction supported accelerators and regions in
#  https://cloud.google.com/vertex-ai/docs/predictions/configure-compute.

# Sets L4 to deploy ImageBind.
machine_type = "g2-standard-8"
accelerator_type = "NVIDIA_L4"
accelerator_count = 1

# Sets V100 to deploy ImageBind.
# machine_type = "n1-standard-8"
# accelerator_type = "NVIDIA_TESLA_V100"
# accelerator_count = 1

model, endpoint = deploy_model(
    model_name=get_job_name_with_datetime(prefix="ImageBind-serve"),
    service_account=SERVICE_ACCOUNT,
    task=task,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
)
print(f"Endpoint name: {endpoint.name}")

注意：部署成功后，预构建的模型权重将在部署后动态下载。因此，在上述模型部署步骤成功之后，需要额外等待5分钟才能运行下面的步骤。否则，在向端点发送请求时，可能会出现“ServiceUnavailable: 503 502:Bad Gateway”错误。

一旦部署成功，您可以使用文本提示、GCS图像路径、深度、热像、视频和音频数据文件以及IMU数据发送请求到端点。

输入规范如下：
- **文本**：由键“text”索引的文本提示列表
- **图像**：由键“vision”索引的3通道RGB图像的GCS路径列表
- **深度图像**：由键“depth”索引的1通道深度图像的GCS路径列表
- **热像图像**：由键“thermal”索引的1通道热像图像的GCS路径列表
- **视频**：由键“video”索引的3通道RGB视频的GCS路径列表
- **音频**：由键“audio”索引的波形文件的GCS路径列表
- **IMU**：由键“imu”索引的形状为[6, 2000]的张量列表（第一维对应于沿X、Y、Z轴的加速计和陀螺仪测量；第二维对应于采样率为200Hz的10秒剪辑）

请参见下面的示例。

In [ ]:
# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint.name` allows us to get the endpoint name of
#   the endpoint `endpoint` created in the cell above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = endpoint.name
# # endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint = aiplatform.Endpoint(aip_endpoint_name)


# If you encounter the issue like `ServiceUnavailable: 503 Took too long to respond when processing`,
# you can reduce the amount of input data.
instances = [
    {
        "text": ["A dog.", "A car", "A bird"],
        "vision": [
            os.path.join(DATA_BUCKET, "dog_image.jpg"),
            os.path.join(DATA_BUCKET, "car_image.jpg"),
            os.path.join(DATA_BUCKET, "bird_image.jpg"),
        ],
        "audio": [
            os.path.join(DATA_BUCKET, "dog_audio.wav"),
            os.path.join(DATA_BUCKET, "car_audio.wav"),
            os.path.join(DATA_BUCKET, "bird_audio.wav"),
        ],
    },
]
response = endpoint.predict(instances=instances)

for modality, embedding in response.predictions[0].items():
    print(f"Modality {modality}: embedding shape {np.array(embedding).shape}")

清理资源

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete model.
model.delete()

### 部署预先构建的ImageBind模型进行零样本分类

在这一部分中，我们部署了一个ImageBind模型，用于在数据模态之间进行零样本分类。

ImageBind模型的峰值GPU内存使用量约为8G。请根据需要调整机器类型、加速器类型和加速器数量。我们在部署中使用一个L4（24G）作为示例。

In [ ]:
task = "zero-shot-classification"

In [ ]:
# Finds Vertex AI prediction supported accelerators and regions in
#  https://cloud.google.com/vertex-ai/docs/predictions/configure-compute.

# Sets L4 to deploy ImageBind.
machine_type = "g2-standard-8"
accelerator_type = "NVIDIA_L4"
accelerator_count = 1

# Sets V100 to deploy ImageBind.
# machine_type = "n1-standard-8"
# accelerator_type = "NVIDIA_TESLA_V100"
# accelerator_count = 1

model, endpoint = deploy_model(
    model_name=get_job_name_with_datetime(prefix="ImageBind-serve"),
    service_account=SERVICE_ACCOUNT,
    task=task,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
)
print(f"Endpoint name: {endpoint.name}")

注意：在部署成功后，预建的模型权重将通过实时下载。因此，在上述模型部署步骤成功之后，需要额外等待**5分钟**，然后才能运行下面的下一个步骤。否则，当您向端点发送请求时，可能会看到`ServiceUnavailable: 503 502:Bad Gateway`错误。

一旦部署成功，您可以向端点发送请求，其中包括文本提示、图像、深度、热像、视频和音频数据文件的GCS路径，以及IMU数据。

输入规范如下：
- **文本**：按“text”键索引的文本提示列表
- **图像**：按“vision”键索引的GCS路径列表，指向3通道RGB图像
- **深度图像**：按“depth”键索引的GCS路径列表，指向1通道深度图像
- **热像图像**：按“thermal”键索引的GCS路径列表，指向1通道热像图像
- **视频**：按“video”键索引的GCS路径列表，指向3通道RGB视频
- **音频**：按“audio”键索引的GCS路径列表，指向波形文件
- **IMU**：按“imu”键索引的形状为[6，2000]的张量列表（第一个维度对应于X、Y、Z轴上的加速计和陀螺仪测量，第二个维度对应于10秒片段，采样率为200Hz）

请参考下面的示例。

In [ ]:
# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint.name` allows us to get the endpoint name of
#   the endpoint `endpoint` created in the cell above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = endpoint.name
# # endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint = aiplatform.Endpoint(aip_endpoint_name)


# If you encounter the issue like `ServiceUnavailable: 503 Took too long to respond when processing`,
# you can reduce the amount of input data.
instances = [
    {
        "text": ["A dog.", "A car", "A bird"],
        "vision": [
            os.path.join(DATA_BUCKET, "dog_image.jpg"),
            os.path.join(DATA_BUCKET, "car_image.jpg"),
            os.path.join(DATA_BUCKET, "bird_image.jpg"),
        ],
        "audio": [
            os.path.join(DATA_BUCKET, "dog_audio.wav"),
            os.path.join(DATA_BUCKET, "car_audio.wav"),
            os.path.join(DATA_BUCKET, "bird_audio.wav"),
        ],
    },
]
response = endpoint.predict(instances=instances)

for modality_pair, probs in response.predictions[0].items():
    print(f"{modality_pair}:\n{np.array(probs)}\n")

清理资源

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete model.
model.delete()